__This contains the master controls and dashboards for all tools. No development should be done herein__

todo

    Summary output should indicate how many new submissions since last run
    
    Update the run multiple tools to use loop when run from command line.
    
    ability to easily get discussion post counts (to troubleshoot reviews)

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
}

In [ ]:
# %cd ~/Dropbox/CanvasHacks
%cd ~/Documents/CanvasHacks
from CanvasHacks import environment

import pandas as pd
pd.options.display.max_rows = 999
from CanvasHacks.Widgets.SkaaControls import multiple_unit_control
from CanvasHacks.Widgets.DashboardControls import button_area, make_control_store
from CanvasHacks.Displays.dashboard import ControlStore
# 
# individ_control_store = make_control_store()
# button_area(individ_control_store)


In [ ]:
environment.__dict__

In [ ]:
from CanvasHacks.executables.easy_skaa_runner import run_multiple_units

control_store = ControlStore()
START_UNIT = 2
STOP_UNIT = 4  
# MINUTES_REST = 120
MINUTES_REST = 45

REPORT_GEN_TIMEOUT = 25#60
REPORT_ID_ATTEMPTS = 100

In [ ]:
try:
    run_multiple_units(control_store, START_UNIT, STOP_UNIT, MINUTES_REST, rest_timeout=REPORT_GEN_TIMEOUT,max_id_attempts=REPORT_ID_ATTEMPTS  )
except:
    run_multiple_units(control_store, START_UNIT, STOP_UNIT, MINUTES_REST, rest_timeout=REPORT_GEN_TIMEOUT,max_id_attempts=REPORT_ID_ATTEMPTS  )
        

In [ ]:
def runner(control_store, start_unit=START_UNIT, stop_unit=STOP_UNIT, minutes_rest=MINUTES_REST, rest_timeout=REPORT_GEN_TIMEOUT,max_id_attempts=REPORT_ID_ATTEMPTS ):
    try:
        run_multiple_units(control_store, start_unit, stop_unit, minutes_rest, rest_timeout, max_id_attempts )
    except:
        runner(control_store, start_unit, stop_unit, minutes_rest, rest_timeout, max_id_attempts )

runner(control_store, start_unit=START_UNIT, stop_unit=STOP_UNIT, minutes_rest=MINUTES_REST, rest_timeout=REPORT_GEN_TIMEOUT,max_id_attempts=REPORT_ID_ATTEMPTS )


In [ ]:
def t():
    try:
        raise Exception
    except:
        print('j')
        t()

t()

In [ ]:
control_store = ControlStore()
multiple_unit_control(control_store)

Remember: for F20 Unit 1 need to update the downloaded file as follows:

    Rename question field to 'body'

    Rename id to student_id
    
    Make a copy of student_id column and call it user_id
    

In [ ]:
# current run
control_store.display_stats()

In [ ]:
# prev run
control_store.d

In [ ]:
control_store.display_incomplete_tables()

# Plots

In [ ]:
#  control_store._get_unit_run_data(8)

In [ ]:
units = [i for i in range(1,7)]
units.append(8)
# print(units)
d = pd.DataFrame([ control_store._get_unit_run_data(i) for i in units])

renames = {
    'unit_number' : 'Unit',
    'essay' : 'Essay',
    'no_essay' : "No Essay",
    'skaa_review' : 'Review',
    'skaa_no_review' : "No Review",
    'skaa_metareview' : "Metareview",
    'skaa_no_metareview' : 'No Metareview'
}

d.rename(renames, axis=1, inplace=True)
# d

def plot_summary(frame, complete_name, incomplete_name):

    TOOLTIPS = [
    #     ("avg total", "@total_patients{0,0}"),
    #         ("date", "@date")
    ]

    p = figure(
        plot_width=1000,
        plot_height=400,
        title=f"{complete_name} ",
    #     x_axis_type='datetime',
    #            tooltips=TOOLTIPS
    )

    #     bar_data = frame.reset_index()
    #     if trim_pos is not None:
    #         bar_data = bar_data[bar_data[field_name] <= trim_pos]
    #     if trim_neg is not None:
    #         bar_data = bar_data[bar_data[field_name] >= trim_neg]

    source = ColumnDataSource(frame)

    # add a circle renderer with a size, color, and alpha
    # p.circle(x='date', y='new_confirmed_cases', size=10, color="navy", source=source, alpha=0.5)
    p.vbar(
        x='Unit',
        top=complete_name,
        color="green",
        width=0.3,
        source=source,
        alpha=0.8,
        legend_label=complete_name)

    p.vbar(
#         x='Unit',
            x=dodge('Unit', -0.3, range=p.x_range),
        top=incomplete_name,
        color="red",
        width=0.3,
        source=source,
        alpha=0.8,
        legend_label=incomplete_name,
)


# source = ColumnDataSource(data=dict(x=x, counts=counts))
    
    # p.line(
    #     x='date',
    #     y='total_patients',
    #     line_width=5,
    #     color="navy",
    #     source=source2,
    #     alpha=0.3,
    #     legend_label='7-day rolling avg')

    p.legend.location = 'bottom_left'

    show(p)


In [ ]:
d.set_index('unit_number',inplace=True)
d

In [ ]:
essay = d[['essay', 'no_essay']]
review = d[['skaa_review', 'skaa_no_review']]
metareview = d[['skaa_metareview', 'skaa_no_metareview']]


In [ ]:
sns.countplot(d[['Essay', 'No Essay']])

In [ ]:
complete_name = 'Essay'
incomplete_name = 'No Essay'
frame = d.copy()
cols = [complete_name, incomplete_name]
units = frame.Unit.tolist()
x = [ (col, unit) for col in cols for unit in units ]
# counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack


In [ ]:
plot_summary(d, 'Essay', 'No Essay')

plot_summary(d, 'Review', 'No Review')

plot_summary(d, 'Metareview', 'No Metareview')

# When things get submitted

from bokeh.transform import jitter
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral6, Plasma, mpl

MAX_UNIT = 8

units = [u for u in range(1,MAX_UNIT + 1)]
subs = []

for u in units:
    try:
        for i, r in control_store.skaa_repos[u].data.iterrows():
            subs.append({ 'unit' : u , 'submitted' : r.invited_to_review} )
    except (KeyError, AttributeError):
        pass

subs = pd.DataFrame(subs)
subs.dropna(inplace=True)
source = ColumnDataSource(subs)

mapper = linear_cmap(field_name='unit', palette=mpl['Plasma'][7] ,low=min(units) ,high=max(units))

p = figure(
        plot_width=900,
        plot_height=300,
        title=f"Essay submissions",
        x_axis_type='datetime') #,

p.circle(x='submitted', 
         y=jitter('unit', width=0.6, range=p.y_range),
         legend_label='unit', 
         color=mapper, 
         source=source, 
         alpha=0.9)
p.legend.location = 'top_left'

show(p)


todo

ability to see which students are waiting

# Students missing some portion

In [ ]:
control_store.display_incomplete_tables()

In [ ]:
control_store.skaa_repos[3].data

# Students who have completed the assignments

In [ ]:
control_store.display_complete_tables()

In [ ]:
control_store.units[3].components[1].__dict__

In [ ]:
environment.CONFIG.unit.review

# Stats after running for individual units

NB, must manually load tables with button before using

## SKAA

### Students who haven't turned in the Essay / content assignment

In [ ]:
individ_control_store['all_steps'][3]['skaa_dash'].no_essay
# control_store['skaa_dash'].no_essay

### Students whose reviewer hasn't turned in the review

In [ ]:
individ_control_store['skaa_dash'].non_reviewed

### Reviewers whose author hasn't turned in metareview

Here we needd to look at the reviewing field. The 'student' is the person who didn't turn in the metareview

In [ ]:
individ_control_store['skaa_dash'].non_metareviewed

## Discussions
### Students who haven't posted to the forum

In [ ]:
individ_control_store['diss_dash'].non_posters

### Students whose reviewers didn't do the review

In [ ]:
individ_control_store['diss_dash'].non_reviewed

In [ ]:
individ_control_store['diss_dash'].reviewed